# Módulo 4, Projeto 3

Vinícius de Souza Miralhas, 10728289

**Atenção:** basta executar as células uma de cada vez. A interface escolhida para o gráfico é a `qt5`. Clique na janela do gráfico: a posição do mouse será lida e uma trajetória no campo de direções será traçada. O programa saberá lidar com vários cliques em posições iniciais distintas. Para reiniciar o gráfico, bastaria ou apertar a tecla <kbd>R</kbd> ou fechar a janela e executar a célula novamente.

# Fontes

+ [Matplotlib Event Handling](https://matplotlib.org/stable/users/event_handling.html)
+ [Get mouse coordinates in Matplotlib](https://stackoverflow.com/questions/51349959/get-mouse-coordinates-without-clicking-in-matplotlib)
+ [Store mouse click event coordinates](https://stackoverflow.com/questions/25521120/store-mouse-click-event-coordinates-with-matplotlib)
+ [Mouse click coordinates as simply as possible](https://stackoverflow.com/questions/37363755/python-mouse-click-coordinates-as-simply-as-possible)

In [2]:
%matplotlib qt5
import matplotlib.pyplot as plt
import numpy as np

# Figura 1

In [11]:
class FP_fig1():
    
    def __init__(self):
        self.fig, self.ax = plt.subplots()
        self.click_cid = self.fig.canvas.mpl_connect("button_press_event", self.on_click)
        self.key_cid = self.fig.canvas.mpl_connect("key_press_event", self.on_press)
        self.generate_ax()
    
    def generate_ax(self):
        self.ax.clear()
        x = np.linspace(-1,1,21)
        y = np.linspace(-1,1,21)

        X,Y = np.meshgrid(x,y)
        # ATENÇÃO: embora a linha abaixo funcione, o único motivo pelo qual funciona é porque são gerados
        # arrays -X e -Y. Se a linha fosse U,V = X,Y, U modificaria X, etc. Sempre use np.copy() OU faça
        # uma operação no array, algo como U,V = 1*X,1*Y.
        U,V = -X,-Y

        # Normaliza U, V
            # np.ndindex produz um excelente generator de loop.
        for i,j in np.ndindex(U.shape):
            if V[i,j] ==0 and U[i,j]==0:
                continue
            # O assignment abaixo DEVE ser simultâneo. U[i,j] e V[i,j] não devem mudar entre os assignments.
            U[i,j],V[i,j] = U[i,j]/np.sqrt(U[i,j]**2+V[i,j]**2), V[i,j]/np.sqrt(U[i,j]**2+V[i,j]**2)

        self.ax.set(title="Clique para simular. Aperte \"r\" para reiniciar.", aspect="equal", 
                    xlim=[-1,1], ylim=[-1,1], xlabel=r"$x$", ylabel=r"$y$")
        self.ax.quiver(X,Y,U,V,color="k",scale=28)
#         self.fig.tight_layout()
        self.fig.canvas.draw()
    
    def numerical_simul(self,x0,y0):
        # Caso queira usar self.numerical_simul ao invpes de FieldPlot.numerical_simul, 
        # adicione self aos args daqui.
        """
        Resolve a seguinte EDO por Euler:
        dx/dt = -x; x(i+1) = x(i) + (-x)*dt
        dy/dt = -y; y(i+1) = y(i) + (-y)*dt
        
        A solução analítica é da forma r = r0 · exp(-t), r,r0 vetores. Logo, x/x0 = exp(-t) = y/y0.
        Portanto, um tempo aceitável de simulação é 10 u.t., que leva a x/x0 = 0,0000454.
        
        Entrada:
        x0 (float64), y0 (float64) :: pos. inicial.
        
        Saída:
        data_array(:,:) (float64) :: array com cols x,y,t.
        """
        x_old,y_old = x0,y0
        t0 = 0     # Define o instante inicial
        t_sim = 10  # Define o tempo de simulação
        dt = 1e-3  # Define o passo temporal; pelo método de Euler (horroroso), são 3 casas decimais de prec.
        data_array = [[x0,y0,t0]] # Define os dados iniciais de simulação
        continue_sim = True # Define se a simulação deve continuar
        
        # Solução Euler
        def euler():
            """
            """
            x_old,y_old,t_old = data_array[-1]
            t = t_old + dt
            
            # Se t > t_sim, pare:
            if t > t_sim:
                return False
            
            # Calcula as pŕoximas coordenadas (lembre-se: o t atual já foi calculado)
            x = x_old + (-x_old)*dt
            y = y_old + (-y_old)*dt

            # Armazena os dados no array:
            data_array.append([x,y,t])
            return True
        
        # A forma de parada está definida em internal_sim
        while continue_sim:
            
            continue_sim = euler()
        
        # Ao encerrar o loop, os dados estão prontos            
        return np.array(data_array)

    
    def on_click(self,event):
        self.ax.scatter(event.xdata, event.ydata ,c="r", marker="x", s=20)
        # Executa a simulação numérica para a curva com condições iniciais dada:
        x0,y0 = event.xdata, event.ydata
        # x0 e/ou y0 podem ser None. Saia da função se o forem:
        if not x0 or not y0: return
        
        # x0 e y0 são válidos:
        x,y,t = self.numerical_simul(x0,y0).T # Unpacking em colunas
        self.ax.plot(x,y,color="maroon")
        self.fig.canvas.draw()
        
    def on_press(self,event):
        key = event.key
        if key=="r":
            self.generate_ax()
    

fp = FP_fig1();

# Figura 3

Figura 2 é irrelevante.

In [14]:
class FP_fig3():
    
    def __init__(self):
        self.fig, self.ax = plt.subplots()
        self.click_cid = self.fig.canvas.mpl_connect("button_press_event", self.on_click)
        self.key_cid = self.fig.canvas.mpl_connect("key_press_event", self.on_press)
        self.generate_ax()
    
    def generate_ax(self):
        """
        Para este caso, plotaremos em (x,t). Portanto, os vetores de velocidade/escoamento serão 
        da forma w = (dx,dt) = dt (dx/dt, 1). Estes vetores serão normalizados ao final e dt é constante, 
        então a forma de w será simplificada para w = (dx/dt,1). Como dx/dt = x, w = (x,1).
        
        Notação: w(i) = [u(i), v(i)]; u(i) = x(i) e v(i) = 1.
        """
        self.ax.clear()
        # O plot será x(t) contra t: ax.quiver(t,x,V,U)
        x = np.linspace(-1,1,21)
        t = np.linspace(-1,1,21)

        X,T = np.meshgrid(x,t)
        # Para as direções dos vetores, usemos:
        u = x
        v = np.ones_like(u)
        U,V = np.meshgrid(u,v)

        # Normaliza U, V
            # np.ndindex produz um excelente generator de loop.
        for i,j in np.ndindex(U.shape):
            if V[i,j] ==0 and U[i,j]==0:
                continue
            # O assignment abaixo DEVE ser simultâneo. U[i,j] e V[i,j] não devem mudar entre os assignments.
            U[i,j],V[i,j] = U[i,j]/np.sqrt(U[i,j]**2+V[i,j]**2), V[i,j]/np.sqrt(U[i,j]**2+V[i,j]**2)

        self.ax.set(title="Clique para simular. Aperte \"r\" para reiniciar.", aspect="equal", 
                    xlim=[-1,1], ylim=[-1,1], xlabel=r"$t$", ylabel=r"$x$")
        self.ax.quiver(T,X,V,U,color="k",scale=28)
#         self.fig.tight_layout()
        self.fig.canvas.draw()
    
    def numerical_simul(self,x0,t0):
        # Caso queira usar self.numerical_simul ao invpes de FieldPlot.numerical_simul, 
        # adicione self aos args daqui.
        """
        Resolve a seguinte EDO por Euler:
        dx/dt = 1*x; x(i+1) = x(i) + x(i)*dt
        
        A solução analítica é da forma x = x0 · exp(t). Logo, x/x0 = exp(t).
        Portanto, o tempo de simulação aqui usado será de 1 u.t., que leva a x/x0 = 2,718. A simulação não
        deve iniciar em um instante qualquer: deverá iniciar no mesmo ponto t0 lido pelo clique.
        
        Entrada:
        x0 (float64) :: pos. inicial.
        
        Saída:
        data_array(:,:) (float64) :: array com cols x,y,t.
        """
        x_old = x0 # Define a pos. inicial
        t_old = t0 # Define o instante inicial
        t_sim = 1  # Define o tempo de simulação
        dt = 1e-3  # Define o passo temporal; pelo método de Euler (horroroso), são 3 casas decimais de prec.
        data_array = [[x0,t0]] # Define os dados iniciais de simulação
        continue_sim = True # Define se a simulação deve continuar
        
        # Solução Euler
        def euler():
            """
            Atua sobre o `data_array` e retorna uma variável lógica para continuar ou parar a simulação.
            """
            x_old,t_old = data_array[-1]
            t = t_old + dt
            
            # Se t > t_sim, pare:
            if t > t_sim:
                return False
            
            # Calcula as pŕoximas coordenadas (lembre-se: o t atual já foi calculado)
            x = x_old + (+x_old)*dt

            # Armazena os dados no array:
            data_array.append([x,t])
            return True
        
        # A forma de parada está definida em internal_sim
        while continue_sim:
            
            continue_sim = euler()
        
        # Ao encerrar o loop, os dados estão prontos            
        return np.array(data_array)

    
    def on_click(self,event):
        self.ax.scatter(event.xdata, event.ydata ,c="r", marker="x", s=20)
        # Executa a simulação numérica para a curva com condições iniciais dada:
        t0,x0 = event.xdata, event.ydata
        # x0 e/ou y0 podem ser None. Saia da função se o forem:
        if not x0 or not t0: return
        
        # x0 e y0 são válidos:
        x,t = self.numerical_simul(x0,t0).T # Unpacking em colunas
        self.ax.plot(t,x,color="maroon")
        self.fig.canvas.draw()
        
    def on_press(self,event):
        key = event.key
        if key=="r":
            self.generate_ax()
    

fp = FP_fig3();

# Figura 4

**Atenção:** para esta figura, deixei o aspect ratio genérico do MPL a fim de aprimorar a visibilidade (ao contrário das outras figuras, onde usei `"equal"`. O problema é que isto impacta nos ângulos corretos do quiver. De modo a restaurar os ângulos certos, faça questão de usar o argumento `.quiver(...,angles="xy")`, o que tornará o gráfico correto independentemente do aspect.

+ [Quiver plot aspect ration](https://stackoverflow.com/questions/12079842/quiver-plot-arrow-aspect-ratio)

In [13]:
class FP_fig4():
    
    def __init__(self):
        self.fig, self.ax = plt.subplots()
        self.click_cid = self.fig.canvas.mpl_connect("button_press_event", self.on_click)
        self.key_cid = self.fig.canvas.mpl_connect("key_press_event", self.on_press)
        self.generate_ax()
        self.auxiliary_curves()
    
    def generate_ax(self):
        """
        Para este caso, plotaremos em (x,t). Portanto, os vetores de velocidade/escoamento serão 
        da forma w = (dx,dt) = dt (dx/dt, 1). Estes vetores serão normalizados ao final e dt é constante, 
        então a forma de w será simplificada para w = (dx/dt,1). Como dx/dt = x-x^2, w = (x-x^2,1).
        
        Notação: w(i) = [u(i), v(i)]; u(i) = x(i) e v(i) = 1.
        """
        self.ax.clear()
        self.auxiliary_curves()
        # O plot será x(t) contra t: ax.quiver(t,x,V,U)
        x = np.linspace(-1,2,31)
        t = np.linspace(-1,1,21) # Usarei 21 pontos para conformar com o plot do documento

        X,T = np.meshgrid(x,t)
        # Para as direções dos vetores, usemos:
        u = x-x**2
        v = np.ones_like(t)
        U,V = np.meshgrid(u,v)

        # Normaliza U, V
            # np.ndindex produz um excelente generator de loop.
        for i,j in np.ndindex(U.shape):
            if V[i,j] ==0 and U[i,j]==0:
                continue
            # O assignment abaixo DEVE ser simultâneo. U[i,j] e V[i,j] não devem mudar entre os assignments.
            U[i,j],V[i,j] = U[i,j]/np.sqrt(U[i,j]**2+V[i,j]**2), V[i,j]/np.sqrt(U[i,j]**2+V[i,j]**2)

        self.ax.set(title="Clique para simular. Aperte \"r\" para reiniciar.",aspect="equal",
                    xlim=[-1,1], ylim=[-1,2], xlabel=r"$t$", ylabel=r"$x$")
        self.ax.quiver(T,X,V,U,color="k",scale=20, angles="xy")
        self.fig.tight_layout()
        self.fig.canvas.draw()
    
    def auxiliary_curves(self):
        t0,x0 = -1,1.0
        x,t = self.numerical_simul(x0,t0).T
        self.ax.plot(t,x,color="midnightblue",linestyle="--")
        
        t0,x0 = -1,0.0
        x,t = self.numerical_simul(x0,t0).T
        self.ax.plot(t,x,color="midnightblue",linestyle="--")        
        self.fig.canvas.draw()
    
    def numerical_simul(self,x0,t0):
        # Caso queira usar self.numerical_simul ao invpes de FieldPlot.numerical_simul, 
        # adicione self aos args daqui.
        """
        Resolve a seguinte EDO por Euler:
        dx/dt = x-x^2; x(i+1) = x(i) + (x(i)-x(i)**2)*dt
        
        A solução analítica é da forma x = x0 · exp(t). Logo, x/x0 = exp(t).
        Portanto, o tempo de simulação aqui usado será de 1 u.t., que leva a x/x0 = 2,718. A simulação não
        deve iniciar em um instante qualquer: deverá iniciar no mesmo ponto t0 lido pelo clique.
        
        Esta simulação lida com o quadrado de x na derivada. As derivadas serão altas e uma precisão por Euler
        de apenas 10^-3 garante baixíssima precisão: usarei 10^-4. Repare que 10^-4 aumentará consideravelmente
        a execução do programa.
        
        Entrada:
        x0 (float64) :: pos. inicial.
        t0 (float64) :: instante inicial
        
        Saída:
        data_array(:,:) (float64) :: array com cols x,t.
        """
        x_old = x0 # Define a pos. inicial
        t_old = t0 # Define o instante inicial
        t_sim = 1  # Define o tempo de simulação
        dt = 1e-4  # Define o passo temporal; pelo método de Euler (horroroso), são 3 casas decimais de prec.
        data_array = [[x0,t0]] # Define os dados iniciais de simulação
        continue_sim = True # Define se a simulação deve continuar
        
        # Solução Euler
        def euler():
            """
            Atua sobre o `data_array` e retorna uma variável lógica para continuar ou parar a simulação.
            """
            x_old,t_old = data_array[-1]
            t = t_old + dt
            
            # Se t > t_sim, pare:
            if t > t_sim:
                return False
            
            # Calcula as pŕoximas coordenadas (lembre-se: o t atual já foi calculado)
            x = x_old + (+x_old-x_old**2)*dt
            
            # Se a derivada for alta o suficiente cancele a simulação. A EDO tem uma descontinuidade inerente.
            if np.abs(x-x**2) > 10:
                return False

            # Armazena os dados no array:
            data_array.append([x,t])
            return True
        
        # A forma de parada está definida em internal_sim
        while continue_sim:
            
            continue_sim = euler()
        
        # Ao encerrar o loop, os dados estão prontos            
        return np.array(data_array)

    
    def on_click(self,event):
        t0,x0 = event.xdata, event.ydata
        # x0 e/ou y0 podem ser None. Saia da função se o forem:
        if not x0 or not t0: return
        
        # x0 e y0 são válidos:
        self.ax.scatter(event.xdata, event.ydata ,c="r", marker="x", s=20)
        
        # Executa a simulação numérica para a curva com condições iniciais dada:
        x,t = self.numerical_simul(x0,t0).T # Unpacking em colunas
        self.ax.plot(t,x,color="maroon")
        self.fig.canvas.draw()
        
    def on_press(self,event):
        key = event.key
        if key=="r":
            self.generate_ax()
    

fp = FP_fig4();

# Figura 5

Baseado na classe da figura 1. Reaproveitei o `angles = "xy"`.

In [15]:
class FP_fig5():
    
    def __init__(self):
        self.fig, self.ax = plt.subplots()
        self.click_cid = self.fig.canvas.mpl_connect("button_press_event", self.on_click)
        self.key_cid = self.fig.canvas.mpl_connect("key_press_event", self.on_press)
        self.generate_ax()
    
    def generate_ax(self):
        self.ax.clear()
        # Gera x,y (e suas versões 2D)
        x = np.linspace(0,4,21)
        y = np.linspace(0,4,21)
        X,Y = np.meshgrid(x,y)
        
        # ATENÇÃO: ISTO NÃO GERA OS U,V CORRETOS! DEVEM SER FEITOS POR LOOP.
        # O motivo? Simples: u,v dependem simultaneamente de x e y e variam na MALHA, não em duas linhas
        # separadas, como acontece nas figuras 3 e 4.
#         # Gera u=dx/dt, v=dy/dt
#         u = x - x*y
#         v = -y + x*y
#         U,V = np.meshgrid(u,v)
        # Este é o loop correto:
#         U,V = np.zeros_like(X),np.zeros_like(Y)
#         for i,j in np.ndindex(U.shape):
#             U[i,j] = X[i,j] - X[i,j]*Y[i,j]
#             V[i,j] = -Y[i,j] + X[i,j]*Y[i,j]
        
        # Uma maneira muito mais direta de alcançar o mesmo resultado é a partir de:
        U = np.copy(X - X*Y)
        V = np.copy(-Y + X*Y)

        # Normaliza U, V
            # np.ndindex produz um excelente generator de loop.
        for i,j in np.ndindex(U.shape):
            if V[i,j] ==0 and U[i,j]==0:
                continue
            # O assignment abaixo DEVE ser simultâneo. U[i,j] e V[i,j] não devem mudar entre os assignments.
            U[i,j],V[i,j] = U[i,j]/np.sqrt(U[i,j]**2+V[i,j]**2), V[i,j]/np.sqrt(U[i,j]**2+V[i,j]**2)

        self.ax.set(title="Clique para simular. Aperte \"r\" para reiniciar.", aspect="equal", 
                    xlim=[-0.02,4], ylim=[-0.02,4], xlabel=r"$x$", ylabel=r"$y$")
        self.ax.quiver(X,Y,U,V,color="k",scale=28,angles="xy")
#         self.fig.tight_layout()
        self.fig.canvas.draw()
    
    def numerical_simul(self,x0,y0):
        # Caso queira usar self.numerical_simul ao invpes de FieldPlot.numerical_simul, 
        # adicione self aos args daqui.
        """
        Resolve a seguinte EDO por Euler:
        dx/dt = x - xy;
        dy/dt = -y + xy;
        
        Entrada:
        x0 (float64), y0 (float64) :: pos. inicial.
        
        Saída:
        data_array(:,:) (float64) :: array com cols x,y,t.
        """
        x_old,y_old = x0,y0
        t0 = 0      # Define o instante inicial
        t_sim = 15  # Define o tempo de simulação. Dado que as curvas devem fechar, usei um tempo longo.
        dt = 1e-3   # Define o passo temporal; pelo método de Euler (horroroso), são 3 casas decimais de prec.
        data_array = [[x0,y0,t0]] # Define os dados iniciais de simulação
        continue_sim = True # Define se a simulação deve continuar
        
        # Solução Euler
        def euler():
            """
            """
            x_old,y_old,t_old = data_array[-1]
            t = t_old + dt
            
            # Se t > t_sim, pare:
            if t > t_sim:
                return False
            
            # Calcula as pŕoximas coordenadas (lembre-se: o t atual já foi calculado)
            x = x_old + (+x_old - x_old*y_old)*dt
            y = y_old + (-y_old + x_old*y_old)*dt

            # Armazena os dados no array:
            data_array.append([x,y,t])
            return True
        
        # A forma de parada está definida em internal_sim
        while continue_sim:
            
            continue_sim = euler()
        
        # Ao encerrar o loop, os dados estão prontos            
        return np.array(data_array)

    
    def on_click(self,event):

        # Executa a simulação numérica para a curva com condições iniciais dada:
        x0,y0 = event.xdata, event.ydata
        # x0 e/ou y0 podem ser None. Saia da função se o forem:
        if not x0 or not y0: return
        
        self.ax.scatter(event.xdata, event.ydata ,c="r", marker="x", s=20)
        # x0 e y0 são válidos:
        x,y,t = self.numerical_simul(x0,y0).T # Unpacking em colunas
        self.ax.plot(x,y,color="maroon")
        self.fig.canvas.draw()
        
    def on_press(self,event):
        key = event.key
        if key=="r":
            self.generate_ax()
    

fp = FP_fig5();

# Figura 6

Considere $y = dx/dt = \dot{x}$. Isto permite plotar a curva de direções no plano $x,y$ a partir das EDOs  
$\dot{x} = y; \ \dot{y} = -x$

In [16]:
class FP_fig6():
    
    def __init__(self):
        self.fig, self.ax = plt.subplots()
        self.click_cid = self.fig.canvas.mpl_connect("button_press_event", self.on_click)
        self.key_cid = self.fig.canvas.mpl_connect("key_press_event", self.on_press)
        self.generate_ax()
    
    def generate_ax(self):
        self.ax.clear()
        # Gera x,y (e suas versões 2D)
        x = np.linspace(-1.05,1.05,21)
        y = np.linspace(-1.05,1.05,21)
        X,Y = np.meshgrid(x,y)
        
        # Uma maneira muito mais direta de alcançar o mesmo resultado é a partir de:
            # Certifique-se de copiar os arrays! Do contrário, U e V modificarão X e Y na normalização.
        U,V = 1*Y,-X

#         Normaliza U, V
#             np.ndindex produz um excelente generator de loop.
        for i,j in np.ndindex(U.shape):
            if V[i,j] ==0 and U[i,j]==0:
                continue
            # O assignment abaixo DEVE ser simultâneo. U[i,j] e V[i,j] não devem mudar entre os assignments.
            U[i,j],V[i,j] = U[i,j]/np.sqrt(U[i,j]**2+V[i,j]**2), V[i,j]/np.sqrt(U[i,j]**2+V[i,j]**2)

        self.ax.set(title="Clique para simular. Aperte \"r\" para reiniciar.", aspect="equal",
                    xlim=[-1,1], ylim=[-1,1], xlabel=r"$x$", ylabel=r"$y = dx/dt$")
        self.ax.quiver(X,Y,U,V,color="k", scale=28)
#         self.fig.tight_layout()
        self.fig.canvas.draw()
    
    def numerical_simul(self,x0,y0):
        # Caso queira usar self.numerical_simul ao invpes de FieldPlot.numerical_simul, 
        # adicione self aos args daqui.
        """
        Resolve a seguinte EDO por Euler:
        dx/dt = y
        dy/dt = -x
        
        Entrada:
        x0 (float64), y0 (float64) :: pos. inicial.
        
        Saída:
        data_array(:,:) (float64) :: array com cols x,y,t.
        """
        x_old,y_old = x0,y0
        t0 = 0      # Define o instante inicial
        t_sim = 10  # Define o tempo de simulação. Dado que as curvas devem fechar, usei um tempo longo.
        dt = 1e-3   # Define o passo temporal; pelo método de Euler (horroroso), são 3 casas decimais de prec.
        data_array = [[x0,y0,t0]] # Define os dados iniciais de simulação
        continue_sim = True # Define se a simulação deve continuar
        
        # Solução Euler
        def euler():
            """
            """
            x_old,y_old,t_old = data_array[-1]
            t = t_old + dt
            
            # Se t > t_sim, pare:
            if t > t_sim:
                return False
            
            # Calcula as pŕoximas coordenadas (lembre-se: o t atual já foi calculado)
            x = x_old + (+y_old)*dt
            y = y_old + (-x_old)*dt

            # Armazena os dados no array:
            data_array.append([x,y,t])
            return True
        
        # A forma de parada está definida em internal_sim
        while continue_sim:
            
            continue_sim = euler()
        
        # Ao encerrar o loop, os dados estão prontos            
        return np.array(data_array)

    
    def on_click(self,event):

        # Executa a simulação numérica para a curva com condições iniciais dada:
        x0,y0 = event.xdata, event.ydata
        # x0 e/ou y0 podem ser None. Saia da função se o forem:
        if not x0 or not y0: return
        
        self.ax.scatter(event.xdata, event.ydata ,c="r", marker="x", s=20)
        # x0 e y0 são válidos:
        x,y,t = self.numerical_simul(x0,y0).T # Unpacking em colunas
        self.ax.plot(x,y,color="maroon")
        self.fig.canvas.draw()
        
    def on_press(self,event):
        key = event.key
        if key=="r":
            self.generate_ax()
    

fp = FP_fig6();

# Figura 7

$\dot{x} = y;\\ \dot{y} = -x -0.4y$

In [5]:
class FP_fig7():
    
    def __init__(self):
        self.fig, self.ax = plt.subplots()
        self.click_cid = self.fig.canvas.mpl_connect("button_press_event", self.on_click)
        self.key_cid = self.fig.canvas.mpl_connect("key_press_event", self.on_press)
        self.generate_ax()
    
    def generate_ax(self):
        self.ax.clear()
        # Gera x,y (e suas versões 2D)
        x = np.linspace(-1.,1.,21)
        y = np.linspace(-1.,1.,21)
        X,Y = np.meshgrid(x,y)
        
        # Uma maneira muito mais direta de alcançar o mesmo resultado é a partir de:
            # Certifique-se de copiar os arrays! Do contrário, U e V modificarão X e Y na normalização.
        U,V = 1*Y,-X -0.4*Y

#         Normaliza U, V
#             np.ndindex produz um excelente generator de loop.
        for i,j in np.ndindex(U.shape):
            if V[i,j] ==0 and U[i,j]==0:
                continue
            # O assignment abaixo DEVE ser simultâneo. U[i,j] e V[i,j] não devem mudar entre os assignments.
            U[i,j],V[i,j] = U[i,j]/np.sqrt(U[i,j]**2+V[i,j]**2), V[i,j]/np.sqrt(U[i,j]**2+V[i,j]**2)

        self.ax.set(title="Clique para simular. Aperte \"r\" para reiniciar.", aspect="equal",
                    xlim=[-1,1], ylim=[-1,1], xlabel=r"$x$", ylabel=r"$y = dx/dt$")
        self.ax.quiver(X,Y,U,V,color="k", scale=28)
#         self.fig.tight_layout()
        self.fig.canvas.draw()
    
    def numerical_simul(self,x0,y0):
        # Caso queira usar self.numerical_simul ao invpes de FieldPlot.numerical_simul, 
        # adicione self aos args daqui.
        """
        Resolve a seguinte EDO por Euler:
        dx/dt = y
        dy/dt = -x -0.4*y
        
        Entrada:
        x0 (float64), y0 (float64) :: pos. inicial.
        
        Saída:
        data_array(:,:) (float64) :: array com cols x,y,t.
        """
        x_old,y_old = x0,y0
        t0 = 0      # Define o instante inicial
        t_sim = 20  # Define o tempo de simulação. Dado que as curvas devem fechar, usei um tempo longo.
        dt = 1e-3   # Define o passo temporal; pelo método de Euler (horroroso), são 3 casas decimais de prec.
        data_array = [[x0,y0,t0]] # Define os dados iniciais de simulação
        continue_sim = True # Define se a simulação deve continuar
        
        # Solução Euler
        def euler():
            """
            """
            x_old,y_old,t_old = data_array[-1]
            t = t_old + dt
            
            # Se t > t_sim, pare:
            if t > t_sim:
                return False
            
            # Calcula as pŕoximas coordenadas (lembre-se: o t atual já foi calculado)
            x = x_old + (+y_old)*dt
            y = y_old + (-x_old-0.4*y_old)*dt

            # Armazena os dados no array:
            data_array.append([x,y,t])
            return True
        
        # A forma de parada está definida em internal_sim
        while continue_sim:
            
            continue_sim = euler()
        
        # Ao encerrar o loop, os dados estão prontos            
        return np.array(data_array)

    
    def on_click(self,event):

        # Executa a simulação numérica para a curva com condições iniciais dada:
        x0,y0 = event.xdata, event.ydata
        # x0 e/ou y0 podem ser None. Saia da função se o forem:
        if not x0 or not y0: return
        
        self.ax.scatter(event.xdata, event.ydata ,c="r", marker="x", s=20)
        # x0 e y0 são válidos:
        x,y,t = self.numerical_simul(x0,y0).T # Unpacking em colunas
        self.ax.plot(x,y,color="maroon")
        self.fig.canvas.draw()
        
    def on_press(self,event):
        key = event.key
        if key=="r":
            self.generate_ax()
    

fp = FP_fig7();